---
title: "Computer Graphics"
author: "Vahram Poghosyan"
date: "2025-05-24"
categories: ["Game Development", "Computer Graphics"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js",
              "OrbitControls": "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js",
              "Cannon": "https://cdn.jsdelivr.net/npm/cannon-es@0.20.0/dist/cannon-es.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-1.js"></script>
    <script type="module" src="./javascript/three-js-basic-shader-demo-2.js"></script>
---

# OpenGL/WebGL & Shaders

OpenGL is a cross-platform graphics API that is widely used in video game development and computer graphics (it's a competitor to Nvidia's DirectX). WebGL is a [language binding](https://en.wikipedia.org/wiki/Language_binding) of OpenGL in JavaScript. I will use WebGL and OpenGL interchangeably throughout these posts. It provides a set of functions for rendering 2D and 3D graphics, allowing developers to create complex visual effects and realistic environments. GLSL is part of OpenGL. In the Three.js post [Three.js: Introduction to 3D Graphics](../../visualization/three_js_in_jupyter/three_js_in_jupyter.ipynb), we learned how to create a simple 3D scene using Three.js, which is a JavaScript library that abstracts the use of OpenGL API. It uses OpenGL's API calls, internally, to compile, link, and send our GLSL shader code to the GPU.

In this post, we will explore how to use shaders in Three.js to create custom visual effects. Shaders are small programs that run on the GPU and are used to control the rendering of graphics. They allow developers to manipulate the appearance of objects in a scene, such as their color, texture, and lighting. Shaders are separate from the [*rendering pipeline*](#the-three-stages-of-the-rendering-pipeline) (see below), they can be thought of as ad-hoc programs that run on the GPU in a massively parallel way. They can run at *any* point of the rendering pipeline, and do so totally independently of it. Because of this, shaders give us fine-grained control over the rendering process.

## Shader Code (GLSL)

As mentioned above, the shader code is written in GLSL (OpenGL Shading Language). The *vertex shader* is responsible for transforming the vertices of the geometry. The fragment shader is responsible for determining the color of each pixel. Note that the fragment shader is also sometimes called *the pixel shader*.

Let's create a new scene containing a simple quadrilateral mesh (created using Three.js, which is an abstraction layer over OpenGL). We will use this quad mesh as our canvas to draw on (with shaders). First, we'll stick to 2D. We'll create a vertex shader that takes the vertices of our quadrelateral and maps them to the full screen. The fragment shader will contain most of the magic, for now. It will essentially paint the surface of the quad... 

<details><summary>Click to expand the code used to create a basic scene</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm';

/* Scene / Camera / renderer ---------------------------------------------- */ 
const bodyWidth = document.getElementById("quarto-document-content").clientWidth;
const bodyHeight = 600;
const canvas = document.getElementById("three-d-canvas");

const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

/* Lights ---------------------------------------------- */
scene.add(new three.AmbientLight(0xffffff, 1));
const dir = new three.DirectionalLight(0xffffff, 1);
dir.position.set(10, 20, 10);
scene.add(dir);

/* Plane  ---------------------------------------------- */
const quadMesh = new three.Mesh(
    new three.PlaneGeometry(250, 250), // Plane is added to the XY plane (x+ = right, y+ = up, z+ = out of the screen)
);
scene.add(quadMesh); 

/* Camera ---------------------------------------------- */
camera.position.set(0, 0, 200); // The z+ represetns 100 units towards the viewer (out of the screen)
camera.lookAt(quadMesh.position);

renderer.render(scene, camera);
```

</details>

Final Result:

In [1]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-1'></canvas>"))

Here it is, our empty canvas. Let's add some color to it. First, we create a vertex shader in the `/shaders` subdirectory of this page. Then, a fragment shader. We import these shaders into our JavaScript code and use them to create a `ShaderMaterial`. Finally, we apply this material to our quad mesh.

```js
import vertexShader from 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.vert?raw';
import fragmentShader from 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/unpublished_posts/game_development/computer_graphics/shaders/shader2.frag?raw';
const shaderMaterial = new three.ShaderMaterial({
    vertexShader: vertexShader,
    fragmentShader: fragmentShader,
    uniforms: {
        uTime: { value: 0.0 },
        uResolution: { value: new three.Vector2(bodyWidth, bodyHeight) }
    }
});

quadMesh.material = shaderMaterial;
```

In [2]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-2'></canvas>"))

# The Three Main Stages of the Rendering Pipeline

After some CPU side pre-processing, the rendering pipeline consists of three main stages:

## Vertex Shading

The first stage of the rendering pipeline is the vertex shading stage. In this stage, the vertex shader is executed for each vertex of the geometry. The vertex shader is responsible for transforming the vertices of the geometry from object space to clip space. It can also be used to perform other operations, such as calculating normals or texture coordinates.

### World Matrix, Local Matrix, and Model Matrix

This is where three crucial matrices come into play: the *model matrix*, the *view matrix*, and the *projection matrix*. The model matrix transforms the vertices from object space to world space, the view matrix transforms the vertices from world space to camera space, and the projection matrix transforms the vertices from camera space to clip space. The final position of each vertex is calculated by multiplying these three matrices together. As always with matrices, the order of multiplication matters. The final position of each vertex is given by:

The position of a given vertex $\mathbf{v}$ in clip space is:

$$
\mathbf{v}_{\text{clip}} = \mathbf{P} \cdot \mathbf{V} \cdot \mathbf{M} \cdot \mathbf{v}
$$

Once the position of each vertex is calculated, a *primitives assembly* stage is performed, where the vertices are grouped into primitives (e.g., triangles, lines, etc.). The primitives are then rasterized to generate fragments, which are the pixels that will be drawn on the screen.

Geometry shading is a more advanced stage that can be used to generate new primitives based on the existing primitives. It is not always used, but it can be useful for certain effects, such as generating shadows or reflections.

A *tessellation stage* can also be used to subdivide the primitives into smaller ones, allowing for more detailed rendering. This is often used in high-end graphics applications, such as video games and simulations.

## Rasterization

The magical step where vector-based primitives are converted into fragments (pixels). This is where the geometry is transformed into a 2D representation that can be displayed on the screen. The rasterization stage takes the primitives generated in the previous stage and converts them into fragments, which are the pixels that will be drawn on the screen. Each fragment is assigned a color based on the lighting and shading calculations performed in the next stage.

Rasterization and *Ray-Tracing* (described briefly in the Three.js post [Three.js: Introduction to 3D Graphics](../../visualization/three_js_in_jupyter/three_js_in_jupyter.ipynb)) are two different approaches. Rasterization is a brute-force and more common approach, while ray-tracing is more accurate and produces more realistic images. Ray-tracing is often used in high-end graphics applications, such as movies and animations, where the quality of the image is more important than the speed of rendering. Rasterization, on the other hand, is used in real-time applications, such as video games, where speed is more important than quality. However, this is changing with the advent of modern GPUs that can perform ray-tracing in real-time.


What's the difference? Rasterization and [ray-tracing](https://en.wikipedia.org/wiki/Ray_tracing_(graphics)) are two different algorithms that both, essentially, take *primitives* (which are vector based) and output *fragments* (which are the last abstraction layer before a pixel color value). The value of a given pixel is determined from the fragment that covers that pixel (and on whether other fragments are in front of *it* and the camera, at least in the case of rasterization). To understand the surface-level difference between rasterization and ray tracing, think in terms of two nested loops. Each object covers a certain area of pixels on the screen, so the rasterization algorithm takes each object first and determines which pixels it covers. Then, for each pixel it determines if the object is the closest one to the given pixel or not. Ray tracing, on the other hand, flips the loops. It asks, for each pixel, which object is the closest one to it. It does this by casting rays from the camera into the scene and checking for intersections with objects. 

Here's a great [video](https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s) from Nvidia that explains the difference between rasterization and ray-tracing in more detail. I will embed it here.

![YouTube: Nvidia's Explanation of Ray Tracing vs Rasterization](https://www.youtube.com/watch?v=ynCxnR1i0QY&t=111s)


Raytracing can be used in conjunction with rasterization to achieve a balance between quality and performance. For example, ray-tracing can be used to calculate reflections and refractions, while rasterization can be used for the rest of the scene. This is often referred to as *hybrid rendering*. For example, *screen-space reflections (SSR)* is a technique that uses ray-tracing to calculate reflections in a scene, while rasterization is used for the rest of the scene. This allows for more realistic reflections without the performance overhead of full ray-tracing. *Ambient occlusion* is another technique that uses ray-tracing to calculate the amount of light that reaches a surface, while rasterization is used for the rest of the scene. While these can be thought of as *post-processing effect* I like to reserve the term *post-processing* for effects that are applied after the entire scene has been drawn and colored (i.e. after *fragment shading*). These effects include: *bloom*, *motion blur*, *depth of field*, *aliasing*, etc. However, after rasterization we already have a colored scene, more or less, so I can see why the term post-processing can still apply. Given that *fragment shading* and *post-processing* are at the same level in the pipeline, it's a matter of preference whether to call these effects post-processing or not. *Post-processing* is *fragment shading* (in essence).

## Fragment Shading

The final stage of the rendering pipeline is the fragment shading stage. In this stage, the fragment shader is executed for each fragment generated in the rasterization stage. The fragment shader is responsible for determining the color of each pixel based on the lighting and shading calculations performed in the previous stages. It can also be used to apply *textures*, perform *post-processing* effects, and more.

A key object in this stage is the *frame buffer*, which is a memory buffer that stores the color and depth information of each pixel. A *Frame Buffer Object (FBO)* is an object that's used to store the shader calculations on a given scene to a texture rather than to the screen itself. This allows us to overlay post-processing effects. Each effect is drawn to its own *texture* within a separate FBO. What's shown on the screen is, then, all of these *textures* (along with other shader calculations performed during the fragment shading stage) overlaid on top of the rasterized scene.

# Euler Angles and Rotation Matrices

## Intertial Frame and Body Frame

$R_{yaw}$, $R_{pitch}$, and $R_{roll}$.

# Quaternion Rotation